In [0]:
root_folder = '/content/drive/My Drive/Colab Notebooks/SMAI/2019201089-Assignment 2/'
train_data_path = root_folder + 'Datasets/Question-5/Train(1).csv'

In [0]:
import os
os.chdir(root_folder)

In [3]:
!ls

 Datasets		        q5_svm.ipynb	      test.py
 q1_cfar10.ipynb	        q6_clustering.ipynb   train.csv
 q3_linear_regression.ipynb     SMAI_A2.pdf	      val.csv
'q4_ linear_regression.ipynb'   test56.py


In [0]:
import pandas as pd
import numpy as np

In [0]:
data_frame = pd.read_csv(train_data_path)

In [6]:
data_frame.head()

Unnamed: 0                                               text author
0           0  The sight of the military restored hope to tho...    MWS
1           1  Just as the building was wiped out by a German...    HPL
2           2                    Do you wonder how it will seem?    HPL
3           3  My revenge is of no moment to you; yet, while ...    MWS
4           4  If he dares to reply, you can tell him from Lu...    EAP

In [0]:
data_frame = data_frame.drop(['Unnamed: 0'],axis=1)

In [8]:
data_frame.head()

text author
0  The sight of the military restored hope to tho...    MWS
1  Just as the building was wiped out by a German...    HPL
2                    Do you wonder how it will seem?    HPL
3  My revenge is of no moment to you; yet, while ...    MWS
4  If he dares to reply, you can tell him from Lu...    EAP

In [9]:
data_frame.author.unique()

array(['MWS', 'HPL', 'EAP'], dtype=object)

###IMPORTS

In [10]:
import nltk
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, r2_score, confusion_matrix
from sklearn import model_selection
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse
from sklearn.svm import SVC
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
class AuthorClassifier:

    def __init__(self, C=1, kernel='rbf', min_df=3, ngram_range=(1,2), stop_words='english', degree=3):
        '''
        Model initialization
        C slack handling term
        kernel type rbf, linear, polynomial
        model holds SVC object
        '''

        self.train_x = self.train_y = None

        '''
        Model parameters
        '''

        self.model = None
        self.C = C
        self.degree=degree
        self.kernel = kernel
        '''
        Preprocessing attrs
        '''
        self.tfidf_vectorizer = None
        self.min_df = min_df
        self.ngram_range = ngram_range
        self.stop_words = 'english'


    def pre_process(self, data_frame):
        '''
        Pre process the data frame
        '''

        class lemma_tokenizer:
            '''
            Lemmathization class to be passed as argument of TFIDF while preprocessing
            '''
            def __init__(self):
                '''
                Used for Wordnet lemmatizer initialization
                '''
                self.word_net_lemmatizer = WordNetLemmatizer()
            def __call__(self, docs):
                return [self.word_net_lemmatizer.lemmatize(token) for token in word_tokenize(docs)]

        text_data = data_frame['text'].values
        print('Doing Pre Processing')
        self.tfidf_vectorizer = TfidfVectorizer(min_df=self.min_df, ngram_range=self.ngram_range, stop_words=self.stop_words, tokenizer=lemma_tokenizer())
        print('\n\n\n', self.tfidf_vectorizer, '\n\n\n')
        self.tfidf_vectorizer.fit(text_data)
        processed_x = sparse.csr_matrix(self.tfidf_vectorizer.transform(text_data).A)
        print('\n\n\n', 'Finished Preprocessing','\n\n\n')
        return processed_x

    def get_support_vectors(self):
        '''
        returns the support vectors
        '''
        return self.model.support_vectors_

    def train(self, train_path='./Datasets/Question-5/Train(1).csv'):
        data_frame = None
        try:
            data_frame = pd.read_csv(train_path).drop(['Unnamed: 0'],axis=1)
            self.train_y = data_frame['author'].values.reshape(-1)
            data_frame = data_frame.drop(['author'], axis=1)
            print('Shape of input ', data_frame.shape, '\n\n\n')
        except:
            print('Error')
        
        if data_frame is not None:
            self.train_x = self.pre_process(data_frame)
            # print('size ',self.train_x.shape, '\n\n\ndata ',self.train_x)
            self.model = SVC(C=self.C,kernel = self.kernel, degree=self.degree)
            print(self.model, '\n\n\n')
            self.model.fit(self.train_x, self.train_y)
        else:
            print('error')



    def predict(self, test_path='./Datasets/Question-5/test.csv'):
        '''
        dont call preprocess instead just do sparse csrmatrix self tfidfvec trasnform if you refit
        absurd result will occur as the train set is you population/sample now
        '''
        test_data_frame = text_test_data = None
        try:
            test_data_frame = pd.read_csv(test_path).drop(['Unnamed: 0'],axis=1)
            text_test_data = test_data_frame['text'].values
        except:
            print('error')
        processed_test_x = None
        if test_data_frame is not None:
            print('\n\nPreprocessing Test set\n\n')
            processed_test_x = sparse.csr_matrix(self.tfidf_vectorizer.transform(text_test_data).A)
        else:
            print('error')
        if processed_test_x is not None:
            return self.model.predict(processed_test_x)
        else:
            print('\n\nprocessed data is none\n\n')
            return None

###Train and test split

In [0]:
data_frame = pd.read_csv(train_data_path)
from sklearn.utils import shuffle
data_frame = shuffle(data_frame)
val=int(-data_frame.shape[0]*0.21)
output=data_frame.iloc[val:]['author']
val_csv = data_frame.iloc[val:].drop(['author'], axis=1)
val_csv.to_csv('val.csv', index=False)
data_frame.iloc[:data_frame.shape[0]+val].to_csv('train.csv', index=False)


In [0]:
dtrain = pd.read_csv('train.csv')

In [14]:
dtrain.head()

Unnamed: 0                                               text author
0         492  I know not how many interminable aeons I sat s...    HPL
1       10624          He repeated the movement a grotesque one.    EAP
2        2446  At first his mother treated his theories as th...    MWS
3        6003  He was, however, exceedingly ugly despite his ...    HPL
4       10475  I was awake, and during the long hours of dead...    MWS

In [0]:
dval = pd.read_csv('val.csv')

In [16]:
dval.head()

Unnamed: 0                                               text
0        4635  The body, as well as the head, was fearfully m...
1        5740  He left me but I still was calm, and after I h...
2        6659  It was clear, therefore, that you would not fa...
3       14978  I exclaimed, "I have now found out the secret"...
4       10086  Their faces expressed a mixture of curiosity a...

###Feature USED
TFIDF
Term frequency: Number of time a word occurs in a document is coined as TF
$$tf = \#word-frequenct$$

INV DOC Freq : log of (number of documents/ total number of occurances of the word in all documnets together)
$$idf = \log(\#documents) / \sum\#tf)$$

###Lemmatization:
Use of lemmatization elimates similar kind of words like working, works all map to work.

###Stop Words Removal
I have remove stopwords which are generally recurring words in english like the, is, this, am, was, were...

### Use of min_df 
Will eliminate word which are less the certain freqency in whole dataset, most probable they are outliers

### RBF kernel

In [17]:
# from q5 import AuthorClassifier as ac
auth_classifier = AuthorClassifier()
auth_classifier.train('train.csv') # Path to the train.csv will be provided
predictions = auth_classifier.predict('val.csv')

Shape of input  (12374, 1) 



Doing Pre Processing



 TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=3, ngram_range=(1, 2), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<__main__.AuthorClassifier.pre_process.<locals>.lemma_tokenizer object at 0x7f53a5155908>,
                use_idf=True, vocabulary=None) 





/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"
/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))





 Finished Preprocessing 



SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False) 





Preprocessing Test set




## Metrics

In [18]:
print('\n\nAccuracy \t', '{0:.5f}'.format(accuracy_score(output, predictions)))

_precision, _recall, _fscore, _support  = precision_recall_fscore_support(output, predictions)

print('precision               \t', ' '.join(["{0:.5f}".format(i) for i in _precision]))
print('recall                  \t',    ' '.join(["{0:.5f}".format(i) for i in _recall]))
print('fscore                  \t',    ' '.join(["{0:.5f}".format(i) for i in _fscore]))
print('Support Vector Count    \t',    auth_classifier.get_support_vectors().shape[0])

print('\n\nConfusion Matrix')
print(confusion_matrix(output, predictions))

print('\n\n\n\n')



Accuracy 	 0.79203
precision               	 0.74250 0.84179 0.82956
recall                  	 0.86419 0.74545 0.74228
fscore                  	 0.79874 0.79070 0.78349
Support Vector Count    	 11513


Confusion Matrix
[[1139   82   97]
 [ 177  697   61]
 [ 218   49  769]]







###Linear Kernel

In [19]:
# from q5 import AuthorClassifier as ac
auth_classifier = AuthorClassifier(kernel='linear')
auth_classifier.train('train.csv') # Path to the train.csv will be provided
predictions = auth_classifier.predict('val.csv')

Shape of input  (12374, 1) 



Doing Pre Processing



 TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=3, ngram_range=(1, 2), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<__main__.AuthorClassifier.pre_process.<locals>.lemma_tokenizer object at 0x7f53a454b7f0>,
                use_idf=True, vocabulary=None) 





/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"
/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))





 Finished Preprocessing 



SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False) 





Preprocessing Test set




## Metrics

In [20]:
print('\n\nAccuracy \t', '{0:.5f}'.format(accuracy_score(output, predictions)))

_precision, _recall, _fscore, _support  = precision_recall_fscore_support(output, predictions)

print('precision               \t', ' '.join(["{0:.5f}".format(i) for i in _precision]))
print('recall                  \t',    ' '.join(["{0:.5f}".format(i) for i in _recall]))
print('fscore                  \t',    ' '.join(["{0:.5f}".format(i) for i in _fscore]))
print('Support Vector Count    \t',    auth_classifier.get_support_vectors().shape[0])

print('\n\nConfusion Matrix')
print(confusion_matrix(output, predictions))

print('\n\n\n\n')



Accuracy 	 0.80420
precision               	 0.77310 0.84131 0.81772
recall                  	 0.85053 0.77112 0.77510
fscore                  	 0.80997 0.80469 0.79584
Support Vector Count    	 9310


Confusion Matrix
[[1121   86  111]
 [ 146  721   68]
 [ 183   50  803]]







##Polynomial Degree 2 kernel

In [21]:
# from q5 import AuthorClassifier as ac
auth_classifier = AuthorClassifier(kernel='poly', degree=2)
auth_classifier.train('train.csv') # Path to the train.csv will be provided
predictions = auth_classifier.predict('val.csv')

Shape of input  (12374, 1) 



Doing Pre Processing



 TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=3, ngram_range=(1, 2), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<__main__.AuthorClassifier.pre_process.<locals>.lemma_tokenizer object at 0x7f539f4bfa20>,
                use_idf=True, vocabulary=None) 





/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"
/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))





 Finished Preprocessing 



SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=2, gamma='scale', kernel='poly',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False) 





Preprocessing Test set




## Metrics

In [22]:
print('\n\nAccuracy \t', '{0:.5f}'.format(accuracy_score(output, predictions)))

_precision, _recall, _fscore, _support  = precision_recall_fscore_support(output, predictions)

print('precision               \t', ' '.join(["{0:.5f}".format(i) for i in _precision]))
print('recall                  \t',    ' '.join(["{0:.5f}".format(i) for i in _recall]))
print('fscore                  \t',    ' '.join(["{0:.5f}".format(i) for i in _fscore]))
print('Support Vector Count    \t',    auth_classifier.get_support_vectors().shape[0])

print('\n\nConfusion Matrix')
print(confusion_matrix(output, predictions))

print('\n\n\n\n')



Accuracy 	 0.77227
precision               	 0.71392 0.82418 0.82809
recall                  	 0.85584 0.72193 0.71139
fscore                  	 0.77847 0.76967 0.76532
Support Vector Count    	 12211


Confusion Matrix
[[1128   87  103]
 [ 210  675   50]
 [ 242   57  737]]







##Polynomial Degree 3 kernel

In [23]:
# from q5 import AuthorClassifier as ac
auth_classifier = AuthorClassifier(kernel='poly', degree=3)
auth_classifier.train('train.csv') # Path to the train.csv will be provided
predictions = auth_classifier.predict('val.csv')

Shape of input  (12374, 1) 



Doing Pre Processing



 TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=3, ngram_range=(1, 2), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<__main__.AuthorClassifier.pre_process.<locals>.lemma_tokenizer object at 0x7f53a51908d0>,
                use_idf=True, vocabulary=None) 





/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"
/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))





 Finished Preprocessing 



SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='poly',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False) 





Preprocessing Test set




## Metrics

In [24]:
print('\n\nAccuracy \t', '{0:.5f}'.format(accuracy_score(output, predictions)))

_precision, _recall, _fscore, _support  = precision_recall_fscore_support(output, predictions)

print('precision               \t', ' '.join(["{0:.5f}".format(i) for i in _precision]))
print('recall                  \t',    ' '.join(["{0:.5f}".format(i) for i in _recall]))
print('fscore                  \t',    ' '.join(["{0:.5f}".format(i) for i in _fscore]))
print('Support Vector Count    \t',    auth_classifier.get_support_vectors().shape[0])

print('\n\nConfusion Matrix')
print(confusion_matrix(output, predictions))

print('\n\n\n\n')



Accuracy 	 0.46032
precision               	 0.42640 0.84524 0.84615
recall                  	 0.97800 0.07594 0.14865
fscore                  	 0.59387 0.13935 0.25287
Support Vector Count    	 12333


Confusion Matrix
[[1289    9   20]
 [ 856   71    8]
 [ 878    4  154]]







In [0]:
# model = auth_classifier.model
# w = model.coef_[0]
# w = w.A[0]
# a = -w[0] / w[1]
# xx = np.linspace(30, 60)
# yy = a * xx - (model.intercept_[0]) / w[1]

# # Plot the parallels to the separating hyperplane that pass through the support vectors
# b = model.support_vectors_[0]
# b = b.A[0]
# yy_down = a * xx + (b[1] - a * b[0])
# b = model.support_vectors_[-1]
# b = b.A[0]
# yy_up = a * xx + (b[1] - a * b[0])

# import seaborn as sns; sns.set(font_scale=1.2)

# print(auth_classifier.train_x.A)

In [0]:
# sns.lmplot(0, 1, data=auth_classifier.train_x.A[0], hue='Type', palette='Set1', fit_reg=False, scatter_kws={"s": 70})
# plt.plot(xx, yy, linewidth=2, color='black')
# plt.plot(xx, yy_down, 'k--')
# plt.plot(xx, yy_up, 'k--')
# plt.scatter(model.support_vectors_[:, 0], model.support_vectors_[:, 1],
#             s=80, facecolors='none');